Importing Libraries

In [1]:
!pip install python_speech_features
!pip install soundfile
!pip install librosa
import os
from scipy.io import wavfile
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Conv2D, MaxPool2D, Flatten, LSTM
from keras.layers import Dropout, Dense, TimeDistributed,BatchNormalization
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
from python_speech_features import mfcc
import pickle
from keras.callbacks import ModelCheckpoint
import pathlib
import librosa
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5887 sha256=83d12c95e864cc491b3c90d2b04c8e0828b476d27be379045cfd49ca1aea5a15
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


Using TensorFlow backend.


Loading and Extracting Data

In [2]:
!wget https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz

--2020-03-08 12:42:13--  https://www.dropbox.com/s/4jw31k5mlzcmgis/genres.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/4jw31k5mlzcmgis/genres.tar.gz [following]
--2020-03-08 12:42:14--  https://www.dropbox.com/s/raw/4jw31k5mlzcmgis/genres.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbbcb0a5162f63eb9f31bf33c7f.dl.dropboxusercontent.com/cd/0/inline/AzgDb7n6ACHe2vyiJXkxW2CdrT5UPGQ13IClFg0z6G2FAm6r-SZSsGANTuETPPzNVgHA6LI6FOQJ-kjZftkgKGQMKYhWKnX12DNSJZa93bpzxQ/file# [following]
--2020-03-08 12:42:14--  https://ucbbcb0a5162f63eb9f31bf33c7f.dl.dropboxusercontent.com/cd/0/inline/AzgDb7n6ACHe2vyiJXkxW2CdrT5UPGQ13IClFg0z6G2FAm6r-SZSsGANTuETPPzNVgHA6LI6FOQJ-kjZftkgKGQMKYhWKnX12DNSJZa93bpzxQ/file
Resolving ucbbcb

In [3]:
!tar -xvf /content/genres.tar.gz

genres/
genres/blues/
genres/blues/blues.00000.au
genres/blues/blues.00001.au
genres/blues/blues.00002.au
genres/blues/blues.00003.au
genres/blues/blues.00004.au
genres/blues/blues.00005.au
genres/blues/blues.00006.au
genres/blues/blues.00007.au
genres/blues/blues.00008.au
genres/blues/blues.00009.au
genres/blues/blues.00010.au
genres/blues/blues.00011.au
genres/blues/blues.00012.au
genres/blues/blues.00013.au
genres/blues/blues.00014.au
genres/blues/blues.00015.au
genres/blues/blues.00016.au
genres/blues/blues.00017.au
genres/blues/blues.00018.au
genres/blues/blues.00019.au
genres/blues/blues.00020.au
genres/blues/blues.00021.au
genres/blues/blues.00022.au
genres/blues/blues.00023.au
genres/blues/blues.00024.au
genres/blues/blues.00025.au
genres/blues/blues.00026.au
genres/blues/blues.00027.au
genres/blues/blues.00028.au
genres/blues/blues.00029.au
genres/blues/blues.00030.au
genres/blues/blues.00031.au
genres/blues/blues.00032.au
genres/blues/blues.00033.au
genres/blues/blues.00034.a

Extracting the Spectogram for all the audio files

In [4]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'/content/genres/{g}'):
        songname = f'/content/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 720x720 with 0 Axes>

In [0]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

Writing Data to csv

In [0]:
import csv

file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'/content/genres/{g}'):
        songname = f'/content/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rmse(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
       
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [7]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00067.au,0.337922,0.129096,2258.538419,2176.031189,4755.429578,0.112765,-95.424423,101.368652,-20.682497,48.655476,-30.258639,20.802527,-18.303758,14.134917,-12.787580,13.433701,-14.072367,7.723721,-19.224149,2.856372,-9.809445,6.911756,-11.303164,8.940704,-2.959639,-0.975428,blues
1,blues.00072.au,0.306928,0.103914,2595.538006,2519.296119,5660.407439,0.129246,-118.665231,87.600960,6.563414,45.094287,-38.903451,20.815969,-26.852990,13.791710,-17.234161,5.104991,-15.705422,3.978413,-27.542309,3.050489,-15.946974,10.868560,-12.646966,2.820748,-13.337095,-4.375539,blues
2,blues.00093.au,0.377690,0.065906,569.930721,995.407125,927.427725,0.021701,-350.436204,169.545752,31.820372,16.682836,28.710147,19.256975,7.863395,10.036764,4.735106,2.907466,3.915553,1.821690,-5.970892,-5.259568,-0.229211,-1.776851,-3.713751,0.181590,2.072390,-2.896225,blues
3,blues.00036.au,0.233230,0.197818,1247.244815,1908.052722,2620.592487,0.036904,-200.220732,116.345181,18.060785,25.288819,0.315825,24.176778,-22.223080,11.183146,-26.841913,-7.643656,-24.872036,-12.015285,-17.009242,-8.115376,-11.959014,-12.617977,-14.187781,-8.204476,-15.024769,-10.322150,blues
4,blues.00020.au,0.302732,0.075387,1389.955510,1911.485152,3005.773491,0.052560,-230.413018,127.094186,7.151100,39.782090,1.951132,17.411689,-5.625576,8.252521,-18.105353,13.839343,-3.763585,2.561243,-4.329452,6.817760,-6.157659,-6.214758,-4.515434,-1.850600,-0.539056,1.508026,blues


In [8]:

data.shape

(1000, 28)

In [0]:
data = data.drop(['filename'],axis=1)

Encoding the Labels

In [0]:

genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

Scaling the Features

In [0]:

scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

Splitting data into train and test sets

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [13]:
len(y_train)

800

In [14]:
len(y_test)

200

Classification

In [15]:

from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [16]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
history = model.fit(X_train,
                    y_train,
                    epochs=100,
                    batch_size=128)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





800/800 [==============================] - 1s 812us/step - loss: 2.1582 - acc: 0.2350
Epoch 2/100
800/800 [==============================] - 0s 34us/step - loss: 1.8285 - acc: 0.3513
Epoch 3/100
800/800 [==============================] - 0s 34us/step - loss: 1.6095 - acc: 0.3950
Epoch 4/100
800/800 [==============================] - 0s 34us/step - loss: 1.4223 - acc: 0.5000
Epoch 5/100
800/800 [==============================] - 0s 39us/step - loss: 1.2753 - acc: 0.5500
Epoch 6/100
800/800 [==============================] - 0s 40us/step - loss: 1.1655 - acc: 0.6000
Epoch 7/100
800/800 [==============================] - 0s 43us/step - loss: 1.0681 - acc: 0.6400
Epoch 8/100
800/800 [==============================] - 0s 38us/step - loss: 1.0016 - acc: 0.6462
Epoch 9/100
800/800 [==============================] - 0s 41us/step - loss: 0.9425 - acc: 0.6775
Epoch 10/100
800/800 [==

In [18]:
test_loss, test_acc = model.evaluate(X_test,y_test)

200/200 [==============================] - 0s 206us/step


In [19]:

print('test_acc: ',test_acc)

test_acc:  0.665


In [0]:

x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [43]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))



model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=100,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Train on 600 samples, validate on 200 samples
Epoch 1/100
600/600 [==============================] - 1s 855us/step - loss: 2.2984 - acc: 0.1133 - val_loss: 2.1664 - val_acc: 0.3900
Epoch 2/100
600/600 [==============================] - 0s 51us/step - loss: 2.1498 - acc: 0.3900 - val_loss: 2.0209 - val_acc: 0.3800
Epoch 3/100
600/600 [==============================] - 0s 53us/step - loss: 2.0002 - acc: 0.3883 - val_loss: 1.8645 - val_acc: 0.3700
Epoch 4/100
600/600 [==============================] - 0s 50us/step - loss: 1.8431 - acc: 0.3833 - val_loss: 1.7367 - val_acc: 0.3800
Epoch 5/100
600/600 [==============================] - 0s 53us/step - loss: 1.7024 - acc: 0.3833 - val_loss: 1.6431 - val_acc: 0.4200
Epoch 6/100
600/600 [==============================] - 0s 58us/step - loss: 1.5719 - acc: 0.4283 - val_loss: 1.5680 - val_acc: 0.4300
Epoch 7/100
600/600 [==============================] - 0s 53us/step - loss: 1.4538 - acc: 0.4767 - val_loss: 1.5095 - val_acc: 0.4700
Epoch 8/100
600

In [44]:
results

[1.7483978271484375, 0.67]

Predictions

In [0]:
predictions = model.predict(X_test)

In [46]:
predictions[0].shape

(10,)

In [47]:
np.sum(predictions[0])

1.0

In [48]:

np.argmax(predictions[0])

9